In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/COVID-19_Radiography_Dataset"
!pip install pip install tensorflow-addons

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 COVID			      Normal.metadata.xlsx
 COVID.metadata.xlsx	      README.md.txt
 Lung_Opacity.metadata.xlsx  'Viral Pneumonia'
 Normal			     'Viral Pneumonia.metadata.xlsx'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras 
import pandas as pd
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense, Reshape, Conv2DTranspose
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import img_to_array
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import math
import numpy as np
import matplotlib.pyplot as plt

from enum import Enum
from glob import glob
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import InstanceNormalization
from PIL import Image


import gdown
from zipfile import ZipFile
# for reproducibility - ref https://machinelearningmastery.com/reproducible-results-neural-networks-keras/ and https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
np.random.seed(9)
tf.keras.utils.set_random_seed(10)

# loading data from gdrive
chest_xray_dataset = os.path.abspath("/content/gdrive/My Drive/COVID 19 CHEST XRAY/images")
chest_xray_dataset_annotations = os.path.abspath("/content/gdrive/My Drive/COVID 19 CHEST XRAY/metadata.csv")
radiography_dataset = os.path.abspath("/content/gdrive/My Drive/COVID-19_Radiography_Dataset/")
xray_covid19_dataset = os.path.abspath("/content/gdrive/My Drive/xray_dataset_covid19/")



In [ ]:
#@markdown #**Anti-Disconnect for Google Colab**
#@markdown ## Run this to stop it from disconnecting automatically 
#@markdown  **(It will anyhow disconnect after 6 - 12 hrs for using the free version of Colab.)**
#@markdown  *(Colab Pro users will get about 24 hrs usage time)*
#@markdown ---
# taken from https://colab.research.google.com/github/justinjohn0306/VQGAN-CLIP/blob/main/VQGAN%2BCLIP_%28z%2Bquantize_method_with_augmentations%2C_user_friendly_interface%29.ipynb#scrollTo=XHyPd4oxVp_l stops colab disconnecting
import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
IPython.display.Javascript(js_code)

<IPython.core.display.Javascript object>

In [ ]:
num_channels = 3
num_classes = 1
latent_dim = 256

In [ ]:
# load images
image_size = (256, 256)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    xray_covid19_dataset + '/Normal/', label_mode=None, image_size=image_size, batch_size=2,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)

Found 94 files belonging to 1 classes.


In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dropout(0),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(2 * 2 * 128),
        layers.Reshape((2, 2, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
             layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
             layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(3, kernel_size=3, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 128, 128, 64)      3136      
                                                                 
 re_lu_40 (ReLU)             (None, 128, 128, 64)      0         
                                                                 
 conv2d_17 (Conv2D)          (None, 64, 64, 128)       131200    
                                                                 
 re_lu_41 (ReLU)             (None, 64, 64, 128)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 32, 32, 128)       262272    
                                                                 
 re_lu_42 (ReLU)             (None, 32, 32, 128)       0         
                                                                 
 flatten_4 (Flatten)         (None, 131072)          

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_RADIOLOGY_Pneumonia')


In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_NORMAL' + '/' + "generated_img_%03d_%d.png" % (epoch, i))

In [ ]:
epochs = 47  # In practice, use ~100 epochs


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(epsilon=1e-8),
    g_optimizer=keras.optimizers.Adam(epsilon=1e-8),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs,steps_per_epoch=1, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)
 
# Save the model
generator.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_NORMALModel/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_NORMALModel/Discriminator',save_format='tf')


Epoch 1/47
1/1 [==============================] - 331s 331s/step - d_loss: 0.6892 - g_loss: 0.8417
Epoch 2/47
1/1 [==============================] - 315s 315s/step - d_loss: 0.4975 - g_loss: 6.4788
Epoch 3/47
1/1 [==============================] - 320s 320s/step - d_loss: 1.2337 - g_loss: 0.4354
Epoch 4/47
1/1 [==============================] - 311s 311s/step - d_loss: 0.7204 - g_loss: 0.4399
Epoch 5/47
1/1 [==============================] - 313s 313s/step - d_loss: 0.9063 - g_loss: 1.3169
Epoch 6/47
1/1 [==============================] - 304s 304s/step - d_loss: 0.2555 - g_loss: 2.7727
Epoch 7/47
1/1 [==============================] - 325s 325s/step - d_loss: 0.3397 - g_loss: 3.6949
Epoch 8/47
1/1 [==============================] - 314s 314s/step - d_loss: 0.2077 - g_loss: 2.8669
Epoch 9/47
1/1 [==============================] - 308s 308s/step - d_loss: 0.4385 - g_loss: 1.5864
Epoch 10/47
1/1 [==============================] - 314s 314s/step - d_loss: 0.4918 - g_loss: 2.8233
Epoch 11/

In [ ]:
# load images
image_size = (64, 64)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    xray_covid19_dataset + '/Pneumonia/', label_mode=None, image_size=image_size, batch_size=2,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)

Found 94 files belonging to 1 classes.


In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dropout(0),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(1 * 1 * 128),
        layers.Reshape((1, 1, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.Conv2D(3, kernel_size=3, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         (None, 32, 32, 64)        3136      
                                                                 
 re_lu_253 (ReLU)            (None, 32, 32, 64)        0         
                                                                 
 conv2d_121 (Conv2D)         (None, 16, 16, 128)       131200    
                                                                 
 re_lu_254 (ReLU)            (None, 16, 16, 128)       0         
                                                                 
 conv2d_122 (Conv2D)         (None, 8, 8, 128)         262272    
                                                                 
 re_lu_255 (ReLU)            (None, 8, 8, 128)         0         
                                                                 
 flatten_30 (Flatten)        (None, 8192)            

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_RADIOLOGY_Pneumonia')


In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_PNEUMONIA' + '/' + "generated_img_%03d_%d.png" % (epoch, i))

In [ ]:
epochs = 47  # In practice, use ~100 epochs


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(epsilon=1e-8),
    g_optimizer=keras.optimizers.Adam(epsilon=1e-8),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs,steps_per_epoch=1, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)
 
# Save the model
generator.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_PNEUMONIAModel/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/Data_Augmented_xray_covid19_dataset_PNEUMONIAModel/Discriminator',save_format='tf')


Epoch 1/47
1/1 [==============================] - 17s 17s/step - d_loss: 0.6679 - g_loss: 0.5334
Epoch 2/47
1/1 [==============================] - 14s 14s/step - d_loss: 0.6759 - g_loss: 0.9541
Epoch 3/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.5918 - g_loss: 1.2054
Epoch 4/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.5569 - g_loss: 1.0290
Epoch 5/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.5501 - g_loss: 0.5420
Epoch 6/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.5484 - g_loss: 0.4681
Epoch 7/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.9397 - g_loss: 0.3734
Epoch 8/47
1/1 [==============================] - 13s 13s/step - d_loss: 1.0388 - g_loss: 0.4794
Epoch 9/47
1/1 [==============================] - 13s 13s/step - d_loss: 1.0379 - g_loss: 0.5872
Epoch 10/47
1/1 [==============================] - 13s 13s/step - d_loss: 0.9804 - g_loss: 0.6362
Epoch 11/47
1/1 [============